In [2]:
import streamlit as st
import numpy as np
import safetensors as stns
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd


/home/doaa/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-24 20:44:51.335989: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-24 20:44:51.410240: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 20:44:53.215795: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different n

In [8]:
model_path = "./saved_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)




In [9]:
data_path = "data/prompt_injection.csv"
dataset = load_dataset("csv", data_files={"test": data_path})
test_dataset = dataset["test"]



Generating test split: 16011 examples [00:00, 30425.91 examples/s]


In [10]:
def combine_prompts(data):
    return {"text": data["System Prompt"] + " " + data["User Prompt"]}

test_dataset = test_dataset.map(combine_prompts)


Map: 100%|██████████| 16011/16011 [00:02<00:00, 7705.41 examples/s] 


In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

test_dataset = test_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.rename_column("Prompt injection", "labels")
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 16011/16011 [00:06<00:00, 2536.30 examples/s]


In [12]:

trainer = Trainer(model=model)

results = trainer.evaluate(test_dataset)
print("📊 Raw Evaluation Results:", results)

predictions = trainer.predict(test_dataset)
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("\n✅ Final Test Metrics:")
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average='weighted'))
print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

📊 Raw Evaluation Results: {'eval_loss': 2.1393704414367676, 'eval_model_preparation_time': 0.006, 'eval_runtime': 1499.4163, 'eval_samples_per_second': 10.678, 'eval_steps_per_second': 1.335}

✅ Final Test Metrics:
Accuracy : 0.7847105115233277
Precision: 0.7382748195487424
Recall   : 0.7847105115233277
F1 Score : 0.730873284510155
